<a href="https://colab.research.google.com/github/hansong0219/Advanced-DeepLearning-Study/blob/master/AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

오토인코더
------------

오토인코더는 주어진 입력 데이터의 압축 표현을 찾기 위한 심층 신경망 구조이다.
오토인코더는 사람이 레이블을 직접 달지 않아도 데이터만 가지고 코드를 자동으로 학습하기 때문에 비지도 학습 알고리즘으로 분류된다. 

오토인코더의 역할은 입력 분포를 벡터형태를 띠는 저차원의 텐서로 인코딩하고 잠재 표현이나, 코드로 불리는 숨겨진 구조로 근사화 한다. 그런다음 이러한 잠재벡터를 원본 입력을 복원하기 위해 디코더를 통해 디코딩 한다.
이런 이유에서 입력 데이터와 출력 데이터의 차이를 손실함수로 근사화한다. 

아래는 MNIST 데이터의 오토인코더의 예시이다.

In [ ]:
from keras.layers import Dense, input
from keras.layers